In [1]:
#-*- coding:utf-8
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Convolution1D, Conv2D, MaxPooling2D, MaxPooling1D, LSTM, Embedding
from keras.optimizers import SGD
import keras.backend as K
import random
import re
import tensorflow as tf
from keras.layers import *
from keras.models import *
from keras.utils import np_utils
from keras.regularizers import l2
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

config_proto = tf.ConfigProto(log_device_placement=0,allow_soft_placement=0)
config_proto.gpu_options.allow_growth = True
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

Using TensorFlow backend.


In [2]:
X_train = np.load('../../20_Galaxy/ANGRY/2HXT-train.npy')
Y_train = np.load('../../20_Galaxy/ANGRY/2HXT-train_label.npy')

X_valid = np.load('../../20_Galaxy/ANGRY/2HXT-val.npy')
Y_valid = np.load('../../20_Galaxy/ANGRY/2HXT-val_label.npy')

In [3]:
one_hot = preprocessing.OneHotEncoder(sparse = False)
y_train = one_hot.fit_transform(Y_train)
y_valid = one_hot.fit_transform(Y_valid)

/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [4]:
y_train

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]])

In [18]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils
def conv_block(input, nb_filter, dropout_rate=None, weight_decay=1E-4):
    x = Activation('relu')(input)
    x = Convolution2D(nb_filter, (1, 9), kernel_initializer="he_uniform", padding="same", use_bias=False,
                      kernel_regularizer=l2(weight_decay))(x)
    if dropout_rate is not None:
        x = Dropout(dropout_rate)(x)
    return x

def dense_block(x, nb_layers, nb_filter, growth_rate, dropout_rate=None, weight_decay=1E-4):
    concat_axis = 1 if K.image_dim_ordering() == "th" else -1

    feature_list = [x]

    for i in range(nb_layers):
        x = conv_block(x, growth_rate, dropout_rate, weight_decay)
        feature_list.append(x)
        x = Concatenate(axis=concat_axis)(feature_list)
        nb_filter += growth_rate

    return x, nb_filter

def transition_block(input, nb_filter, dropout_rate=None, weight_decay=1E-4):
    concat_axis = 1 if K.image_dim_ordering() == "th" else -1

    x = Convolution2D(nb_filter, (1, 1), kernel_initializer="he_uniform", padding="same", use_bias=False,
                      kernel_regularizer=l2(weight_decay))(input)
    if dropout_rate is not None:
        x = Dropout(dropout_rate)(x)
    x = AveragePooling2D((1, 2), strides=(1, 2))(x)

    x = BatchNormalization(axis=concat_axis, gamma_regularizer=l2(weight_decay),
                           beta_regularizer=l2(weight_decay))(x)

    return x

def createDenseNet(nb_classes, img_dim, depth=40, nb_dense_block=3, growth_rate=12, nb_filter=16, dropout_rate=None,
                     weight_decay=1E-4, verbose=True):

    model_input = Input(shape=img_dim)

    concat_axis = 1 if K.image_dim_ordering() == "th" else -1

    assert (depth - 4) % 3 == 0, "Depth must be 3 N + 4"

    # layers in each dense block
    nb_layers = int((depth - 4) / 3)

    # Initial convolution
    x = Convolution2D(nb_filter, (1, 9), kernel_initializer="he_uniform", padding="same", name="initial_conv2D", use_bias=False,
                      kernel_regularizer=l2(weight_decay))(model_input)

    x = BatchNormalization(axis=concat_axis, gamma_regularizer=l2(weight_decay),
                            beta_regularizer=l2(weight_decay))(x)

    # Add dense blocks
    for block_idx in range(nb_dense_block - 1):
        x, nb_filter = dense_block(x, nb_layers, nb_filter, growth_rate, dropout_rate=dropout_rate,
                                   weight_decay=weight_decay)
        # add transition_block
        x = transition_block(x, nb_filter, dropout_rate=dropout_rate, weight_decay=weight_decay)

    # The last dense_block does not have a transition_block
    x, nb_filter = dense_block(x, nb_layers, nb_filter, growth_rate, dropout_rate=dropout_rate,
                               weight_decay=weight_decay)

    x = Activation('relu')(x)
    x = GlobalAveragePooling2D()(x)
    x = Dense(nb_classes, activation='softmax', kernel_regularizer=l2(weight_decay), bias_regularizer=l2(weight_decay))(x)

    densenet = Model(inputs=model_input, outputs=x)

    if verbose: 
        print("DenseNet-%d-%d created." % (depth, growth_rate))

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 3640
CHANNELS = 1
nb_classes = 2
batch_size = 32
nb_epoch = 40
img_dim = (ROWS,COLS,CHANNELS)
densenet_depth = 40
densenet_growth_rate = 12

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim,depth=densenet_depth,
                  growth_rate = densenet_growth_rate)

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')
xtrain = X_train.reshape(-1,12,3640,1)
xvalid = X_valid.reshape(-1,12,3640,1)
model.fit(xtrain, y_train, batch_size=12, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])

DenseNet-40-12 created.
Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 933s 60ms/step - loss: 0.8302 - acc: 0.6554 - val_loss: 0.7641 - val_acc: 0.7398
Epoch 2/50
15658/15658 [==============================] - 927s 59ms/step - loss: 0.7789 - acc: 0.6979 - val_loss: 0.8154 - val_acc: 0.7255
Epoch 3/50
15658/15658 [==============================] - 924s 59ms/step - loss: 0.7597 - acc: 0.7019 - val_loss: 0.7210 - val_acc: 0.7658
Epoch 4/50
15658/15658 [==============================] - 922s 59ms/step - loss: 0.7371 - acc: 0.7171 - val_loss: 0.7049 - val_acc: 0.7978
Epoch 5/50
15658/15658 [==============================] - 921s 59ms/step - loss: 0.7197 - acc: 0.7237 - val_loss: 0.6813 - val_acc: 0.7615
Epoch 6/50
15658/15658 [==============================] - 920s 59ms/step - loss: 0.6979 - acc: 0.7300 - val_loss: 2.6567 - val_acc: 0.5000
Epoch 7/50
15658/15658 [==============================] - 920s 59ms/step - loss: 0.6859 - acc

In [27]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils
def conv_block(input, nb_filter, dropout_rate=None, weight_decay=1E-4):
    x = Activation('relu')(input)
    x = Convolution2D(nb_filter, (1, 21), kernel_initializer="he_uniform", padding="same", use_bias=False,
                      kernel_regularizer=l2(weight_decay))(x)
    if dropout_rate is not None:
        x = Dropout(dropout_rate)(x)
    return x

def dense_block(x, nb_layers, nb_filter, growth_rate, dropout_rate=None, weight_decay=1E-4):
    concat_axis = 1 if K.image_dim_ordering() == "th" else -1

    feature_list = [x]

    for i in range(nb_layers):
        x = conv_block(x, growth_rate, dropout_rate, weight_decay)
        feature_list.append(x)
        x = Concatenate(axis=concat_axis)(feature_list)
        nb_filter += growth_rate

    return x, nb_filter

def transition_block(input, nb_filter, dropout_rate=None, weight_decay=1E-4):
    concat_axis = 1 if K.image_dim_ordering() == "th" else -1

    x = Convolution2D(nb_filter, (1, 1), kernel_initializer="he_uniform", padding="same", use_bias=False,
                      kernel_regularizer=l2(weight_decay))(input)
    if dropout_rate is not None:
        x = Dropout(dropout_rate)(x)
    x = AveragePooling2D((1, 2), strides=(1, 2))(x)

    x = BatchNormalization(axis=concat_axis, gamma_regularizer=l2(weight_decay),
                           beta_regularizer=l2(weight_decay))(x)

    return x

def createDenseNet(nb_classes, img_dim, depth=40, nb_dense_block=3, growth_rate=12, nb_filter=16, dropout_rate=None,
                     weight_decay=1E-4, verbose=True):

    model_input = Input(shape=img_dim)

    concat_axis = 1 if K.image_dim_ordering() == "th" else -1

    assert (depth - 4) % 3 == 0, "Depth must be 3 N + 4"

    # layers in each dense block
    nb_layers = int((depth - 4) / 3)

    # Initial convolution
    x = Convolution2D(nb_filter, (1, 21), kernel_initializer="he_uniform", padding="same", name="initial_conv2D", use_bias=False,
                      kernel_regularizer=l2(weight_decay))(model_input)

    x = BatchNormalization(axis=concat_axis, gamma_regularizer=l2(weight_decay),
                            beta_regularizer=l2(weight_decay))(x)

    # Add dense blocks
    for block_idx in range(nb_dense_block - 1):
        x, nb_filter = dense_block(x, nb_layers, nb_filter, growth_rate, dropout_rate=dropout_rate,
                                   weight_decay=weight_decay)
        # add transition_block
        x = transition_block(x, nb_filter, dropout_rate=dropout_rate, weight_decay=weight_decay)

    # The last dense_block does not have a transition_block
    x, nb_filter = dense_block(x, nb_layers, nb_filter, growth_rate, dropout_rate=dropout_rate,
                               weight_decay=weight_decay)

    x = Activation('relu')(x)
    x = GlobalAveragePooling2D()(x)
    x = Dense(nb_classes, activation='softmax', kernel_regularizer=l2(weight_decay), bias_regularizer=l2(weight_decay))(x)

    densenet = Model(inputs=model_input, outputs=x)

    if verbose: 
        print("DenseNet-%d-%d created." % (depth, growth_rate))

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 3640
CHANNELS = 1
nb_classes = 2
batch_size = 32
nb_epoch = 40
img_dim = (ROWS,COLS,CHANNELS)
densenet_depth = 40
densenet_growth_rate = 8

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim,depth=densenet_depth,
                  growth_rate = densenet_growth_rate)

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')
xtrain = X_train.reshape(-1,12,3640,1)
xvalid = X_valid.reshape(-1,12,3640,1)
model.fit(xtrain, y_train, batch_size=12, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])

DenseNet-40-8 created.
Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 941s 60ms/step - loss: 0.7439 - acc: 0.6722 - val_loss: 0.7206 - val_acc: 0.7945
Epoch 2/50
15658/15658 [==============================] - 941s 60ms/step - loss: 0.6799 - acc: 0.7253 - val_loss: 0.7019 - val_acc: 0.7370
Epoch 3/50
15658/15658 [==============================] - 940s 60ms/step - loss: 0.6520 - acc: 0.7397 - val_loss: 0.6777 - val_acc: 0.7975
Epoch 4/50
15658/15658 [==============================] - 942s 60ms/step - loss: 0.6269 - acc: 0.7565 - val_loss: 0.6694 - val_acc: 0.8205
Epoch 5/50
15658/15658 [==============================] - 942s 60ms/step - loss: 0.6044 - acc: 0.7698 - val_loss: 0.7361 - val_acc: 0.8093
Epoch 6/50
15658/15658 [==============================] - 937s 60ms/step - loss: 0.5881 - acc: 0.7772 - val_loss: 0.5123 - val_acc: 0.8620
Epoch 7/50
15658/15658 [==============================] - 940s 60ms/step - loss: 0.5719 - acc:

In [8]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils
def conv_block(input, nb_filter, dropout_rate=None, weight_decay=1E-4):
    x = Activation('relu')(input)
    x = Convolution2D(nb_filter, (1, 21), kernel_initializer="he_uniform", padding="same", use_bias=False,
                      kernel_regularizer=l2(weight_decay))(x)
    if dropout_rate is not None:
        x = Dropout(dropout_rate)(x)
    return x

def dense_block(x, nb_layers, nb_filter, growth_rate, dropout_rate=None, weight_decay=1E-4):
    concat_axis = 1 if K.image_dim_ordering() == "th" else -1

    feature_list = [x]

    for i in range(nb_layers):
        x = conv_block(x, growth_rate, dropout_rate, weight_decay)
        feature_list.append(x)
        x = Concatenate(axis=concat_axis)(feature_list)
        nb_filter += growth_rate

    return x, nb_filter

def transition_block(input, nb_filter, dropout_rate=None, weight_decay=1E-4):
    concat_axis = 1 if K.image_dim_ordering() == "th" else -1

    x = Convolution2D(nb_filter, (1, 1), kernel_initializer="he_uniform", padding="same", use_bias=False,
                      kernel_regularizer=l2(weight_decay))(input)
    if dropout_rate is not None:
        x = Dropout(dropout_rate)(x)
    x = AveragePooling2D((1, 2), strides=(1, 2))(x)

    x = BatchNormalization(axis=concat_axis, gamma_regularizer=l2(weight_decay),
                           beta_regularizer=l2(weight_decay))(x)

    return x

def createDenseNet(nb_classes, img_dim, depth=40, nb_dense_block=3, growth_rate=12, nb_filter=16, dropout_rate=None,
                     weight_decay=1E-4, verbose=True):

    model_input = Input(shape=img_dim)

    concat_axis = 1 if K.image_dim_ordering() == "th" else -1

    assert (depth - 4) % 3 == 0, "Depth must be 3 N + 4"

    # layers in each dense block
    nb_layers = int((depth - 4) / 3)

    # Initial convolution
    x = Convolution2D(nb_filter, (1, 21), kernel_initializer="he_uniform", padding="same", name="initial_conv2D", use_bias=False,
                      kernel_regularizer=l2(weight_decay))(model_input)

    x = BatchNormalization(axis=concat_axis, gamma_regularizer=l2(weight_decay),
                            beta_regularizer=l2(weight_decay))(x)

    # Add dense blocks
    for block_idx in range(nb_dense_block - 1):
        x, nb_filter = dense_block(x, nb_layers, nb_filter, growth_rate, dropout_rate=dropout_rate,
                                   weight_decay=weight_decay)
        # add transition_block
        x = transition_block(x, nb_filter, dropout_rate=dropout_rate, weight_decay=weight_decay)

    # The last dense_block does not have a transition_block
    x, nb_filter = dense_block(x, nb_layers, nb_filter, growth_rate, dropout_rate=dropout_rate,
                               weight_decay=weight_decay)

    x = Activation('relu')(x)
    x = GlobalAveragePooling2D()(x)
    x = Dense(nb_classes, activation='softmax', kernel_regularizer=l2(weight_decay), bias_regularizer=l2(weight_decay))(x)

    densenet = Model(inputs=model_input, outputs=x)

    if verbose: 
        print("DenseNet-%d-%d created." % (depth, growth_rate))

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 3640
CHANNELS = 1
nb_classes = 2
batch_size = 12
nb_epoch = 40
img_dim = (ROWS,COLS,CHANNELS)
densenet_depth = 40
densenet_growth_rate = 8

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim,depth=densenet_depth,
                  growth_rate = densenet_growth_rate)

sgd = SGD(lr=0.005, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='auto')
xtrain = X_train.reshape(-1,12,3640,1)
xvalid = X_valid.reshape(-1,12,3640,1)
model.fit(xtrain, y_train, batch_size=12, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])

DenseNet-40-8 created.
Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 917s 59ms/step - loss: 0.7422 - acc: 0.6827 - val_loss: 0.6711 - val_acc: 0.7595
Epoch 2/50
15658/15658 [==============================] - 918s 59ms/step - loss: 0.6803 - acc: 0.7279 - val_loss: 0.7303 - val_acc: 0.7645
Epoch 3/50
15658/15658 [==============================] - 917s 59ms/step - loss: 0.6629 - acc: 0.7408 - val_loss: 0.6969 - val_acc: 0.7673
Epoch 4/50
15658/15658 [==============================] - 920s 59ms/step - loss: 0.6383 - acc: 0.7576 - val_loss: 0.5879 - val_acc: 0.8315
Epoch 5/50
15658/15658 [==============================] - 911s 58ms/step - loss: 0.6125 - acc: 0.7726 - val_loss: 0.5980 - val_acc: 0.8320
Epoch 6/50
15658/15658 [==============================] - 917s 59ms/step - loss: 0.5932 - acc: 0.7862 - val_loss: 0.6111 - val_acc: 0.8410
Epoch 7/50
15658/15658 [==============================] - 913s 58ms/step - loss: 0.5721 - acc:

In [11]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils
def conv_block(input, nb_filter, dropout_rate=None, weight_decay=1E-4):
    x = Activation('relu')(input)
    x = Convolution2D(nb_filter, (1, 21), kernel_initializer="he_uniform", padding="same", use_bias=False,
                      kernel_regularizer=l2(weight_decay))(x)
    if dropout_rate is not None:
        x = Dropout(dropout_rate)(x)
    return x

def dense_block(x, nb_layers, nb_filter, growth_rate, dropout_rate=None, weight_decay=1E-4):
    concat_axis = 1 if K.image_dim_ordering() == "th" else -1

    feature_list = [x]

    for i in range(nb_layers):
        x = conv_block(x, growth_rate, dropout_rate, weight_decay)
        feature_list.append(x)
        x = Concatenate(axis=concat_axis)(feature_list)
        nb_filter += growth_rate

    return x, nb_filter

def transition_block(input, nb_filter, dropout_rate=None, weight_decay=1E-4):
    concat_axis = 1 if K.image_dim_ordering() == "th" else -1

    x = Convolution2D(nb_filter, (1, 1), kernel_initializer="he_uniform", padding="same", use_bias=False,
                      kernel_regularizer=l2(weight_decay))(input)
    if dropout_rate is not None:
        x = Dropout(dropout_rate)(x)
    x = AveragePooling2D((1, 2), strides=(1, 2))(x)

    x = BatchNormalization(axis=concat_axis, gamma_regularizer=l2(weight_decay),
                           beta_regularizer=l2(weight_decay))(x)

    return x

def createDenseNet(nb_classes, img_dim, depth=40, nb_dense_block=3, growth_rate=12, nb_filter=16, dropout_rate=None,
                     weight_decay=1E-4, verbose=True):

    model_input = Input(shape=img_dim)

    concat_axis = 1 if K.image_dim_ordering() == "th" else -1

    assert (depth - 4) % 3 == 0, "Depth must be 3 N + 4"

    # layers in each dense block
    nb_layers = int((depth - 4) / 3)

    # Initial convolution
    x = Convolution2D(nb_filter, (1, 21), kernel_initializer="he_uniform", padding="same", name="initial_conv2D", use_bias=False,
                      kernel_regularizer=l2(weight_decay))(model_input)

    x = BatchNormalization(axis=concat_axis, gamma_regularizer=l2(weight_decay),
                            beta_regularizer=l2(weight_decay))(x)

    # Add dense blocks
    for block_idx in range(nb_dense_block - 1):
        x, nb_filter = dense_block(x, nb_layers, nb_filter, growth_rate, dropout_rate=dropout_rate,
                                   weight_decay=weight_decay)
        # add transition_block
        x = transition_block(x, nb_filter, dropout_rate=dropout_rate, weight_decay=weight_decay)

    # The last dense_block does not have a transition_block
    x, nb_filter = dense_block(x, nb_layers, nb_filter, growth_rate, dropout_rate=dropout_rate,
                               weight_decay=weight_decay)

    x = Activation('relu')(x)
    x = GlobalAveragePooling2D()(x)
    x = Dense(nb_classes, activation='softmax', kernel_regularizer=l2(weight_decay), bias_regularizer=l2(weight_decay))(x)

    densenet = Model(inputs=model_input, outputs=x)

    if verbose: 
        print("DenseNet-%d-%d created." % (depth, growth_rate))

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 3640
CHANNELS = 1
nb_classes = 2
batch_size = 12
nb_epoch = 40
img_dim = (ROWS,COLS,CHANNELS)
densenet_depth = 40
densenet_growth_rate = 10

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim,depth=densenet_depth,
                  growth_rate = densenet_growth_rate)

sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, verbose=1, mode='auto')
xtrain = X_train.reshape(-1,12,3640,1)
xvalid = X_valid.reshape(-1,12,3640,1)
model.fit(xtrain, y_train, batch_size=12, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])

DenseNet-40-10 created.
Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 1142s 73ms/step - loss: 0.7970 - acc: 0.6624 - val_loss: 1.0330 - val_acc: 0.6465
Epoch 2/50
15658/15658 [==============================] - 1143s 73ms/step - loss: 0.7361 - acc: 0.7155 - val_loss: 0.7228 - val_acc: 0.8063
Epoch 3/50
15658/15658 [==============================] - 1148s 73ms/step - loss: 0.7119 - acc: 0.7325 - val_loss: 0.6772 - val_acc: 0.8150
Epoch 4/50
15658/15658 [==============================] - 1149s 73ms/step - loss: 0.6929 - acc: 0.7497 - val_loss: 0.7958 - val_acc: 0.7313
Epoch 5/50
15658/15658 [==============================] - 1145s 73ms/step - loss: 0.6739 - acc: 0.7565 - val_loss: 0.6723 - val_acc: 0.8265
Epoch 6/50
15658/15658 [==============================] - 1146s 73ms/step - loss: 0.6547 - acc: 0.7739 - val_loss: 0.6267 - val_acc: 0.8045
Epoch 7/50
15658/15658 [==============================] - 1144s 73ms/step - loss: 0.628

In [12]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils

def cut(x, index):
    x = x[:,index]
    return x

def createDenseNet(nb_classes, img_dim):

    feature_list = []
    
    model_input = Input(shape=img_dim)
    
    for i in range(ROWS):
        inputs = Lambda(cut, arguments={'index':i})(model_input)
#         print inputs
#         print model_input

        cnn1 = Convolution1D(48, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(inputs)
        cnn2 = Convolution1D(48, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn1)
        pool1 = MaxPooling1D(pool_size=(7))(cnn2)
        drop1 = Dropout(0.25)(pool1)

#         drop1 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop1)
        
        cnn3 = Convolution1D(56, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop1)
        cnn4 = Convolution1D(56, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn3)
        pool2 = MaxPooling1D(pool_size=(6))(cnn4)
        drop2 = Dropout(0.25)(pool2)
        
#         drop2 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop2)
        
        cnn5 = Convolution1D(40, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop2)
        cnn6 = Convolution1D(40, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn5)
        pool3 = MaxPooling1D(pool_size=(6))(cnn6)
        drop3 = Dropout(0.25)(pool3)
        
#         print drop3
        feature_list.append(drop3)
        
    concat = Concatenate(axis=2)(feature_list)
    print concat
    
    lstm_out1 = LSTM(64,return_sequences=True)(concat)
    
#     flatten1 = Attention_layer(lstm_out1)
    
    flatten1 = Flatten()(lstm_out1)
    
    x = Dense(nb_classes, kernel_initializer='glorot_uniform', activation='softmax')(flatten1)

    densenet = Model(inputs=model_input, outputs=x)

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 3640
CHANNELS = 1
nb_classes = 2
batch_size = 32
nb_epoch = 40
img_dim = (ROWS,COLS,CHANNELS)

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim)

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_acc', patience=5, verbose=1, mode='auto')
xtrain = X_train.reshape(-1,12,3640,1)
xvalid = X_valid.reshape(-1,12,3640,1)
model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])

/usr/local/lib/python2.7/dist-packages/keras/activations.py:211: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


Tensor("concatenate_259/concat:0", shape=(?, 10, 480), dtype=float32)
Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 115s 7ms/step - loss: 0.6069 - acc: 0.6500 - val_loss: 0.4529 - val_acc: 0.8217
Epoch 2/50
15658/15658 [==============================] - 103s 7ms/step - loss: 0.4697 - acc: 0.7731 - val_loss: 0.4463 - val_acc: 0.8605
Epoch 3/50
15658/15658 [==============================] - 105s 7ms/step - loss: 0.4135 - acc: 0.8106 - val_loss: 0.4503 - val_acc: 0.8698
Epoch 4/50
15658/15658 [==============================] - 106s 7ms/step - loss: 0.3823 - acc: 0.8303 - val_loss: 0.4869 - val_acc: 0.8867
Epoch 5/50
15658/15658 [==============================] - 106s 7ms/step - loss: 0.3686 - acc: 0.8389 - val_loss: 0.4642 - val_acc: 0.8992
Epoch 6/50
15658/15658 [==============================] - 106s 7ms/step - loss: 0.3573 - acc: 0.8458 - val_loss: 0.4401 - val_acc: 0.8902
Epoch 7/50
15658/15658 [=============================

In [23]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils

def cut(x, index):
    x = x[:,index]
    return x

def createDenseNet(nb_classes, img_dim):

    feature_list = []
    
    model_input = Input(shape=img_dim)
    
    for i in range(3):
#         inputs = Lambda(cut, arguments={'index':i})(model_input)
#         print inputs
#         print model_input

        cnn1 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(model_input)
        cnn2 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn1)
        pool1 = MaxPooling1D(pool_size=(7))(cnn2)
        drop1 = Dropout(0.25)(pool1)

#         drop1 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop1)
        
        cnn3 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop1)
        cnn4 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn3)
        pool2 = MaxPooling1D(pool_size=(6))(cnn4)
        drop2 = Dropout(0.25)(pool2)
        
#         drop2 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop2)
        
        cnn5 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop2)
        cnn6 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn5)
        pool3 = MaxPooling1D(pool_size=(6))(cnn6)
        drop3 = Dropout(0.25)(pool3)
        
#         print drop3
        feature_list.append(drop3)
        
    concat = Concatenate(axis=1)(feature_list)
    print concat
    
    lstm_out1 = LSTM(64,return_sequences=True)(concat)
    
#     flatten1 = Attention_layer(lstm_out1)
    
    flatten1 = Flatten()(lstm_out1)
    
    x = Dense(nb_classes, kernel_initializer='glorot_uniform', activation='softmax')(flatten1)

    densenet = Model(inputs=model_input, outputs=x)

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 3640
CHANNELS = 1
nb_classes = 2
batch_size = 32
nb_epoch = 40
img_dim = (ROWS*COLS,CHANNELS)

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim)

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_acc', patience=5, verbose=1, mode='auto')
xtrain = X_train.reshape(-1,12*3640,1)
xvalid = X_valid.reshape(-1,12*3640,1)
model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])

Tensor("concatenate_267/concat:0", shape=(?, 507, 30), dtype=float32)
Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 633s 40ms/step - loss: 0.5566 - acc: 0.7000 - val_loss: 0.4535 - val_acc: 0.8482
Epoch 2/50
15658/15658 [==============================] - 611s 39ms/step - loss: 0.4418 - acc: 0.7940 - val_loss: 0.4424 - val_acc: 0.8642
Epoch 3/50
15658/15658 [==============================] - 612s 39ms/step - loss: 0.4237 - acc: 0.8046 - val_loss: 0.4328 - val_acc: 0.8682
Epoch 4/50
15658/15658 [==============================] - 611s 39ms/step - loss: 0.3799 - acc: 0.8317 - val_loss: 0.4040 - val_acc: 0.8980
Epoch 5/50
15658/15658 [==============================] - 610s 39ms/step - loss: 0.3572 - acc: 0.8469 - val_loss: 0.4547 - val_acc: 0.8640
Epoch 6/50
15658/15658 [==============================] - 606s 39ms/step - loss: 0.3450 - acc: 0.8534 - val_loss: 0.3834 - val_acc: 0.9045
Epoch 7/50
15658/15658 [=======================

In [25]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils

def cut(x, index):
    x = x[:,index]
    return x

def createDenseNet(nb_classes, img_dim):

    feature_list = []
    
    model_input = Input(shape=img_dim)
    
    for i in range(2):
#         inputs = Lambda(cut, arguments={'index':i})(model_input)
#         print inputs
#         print model_input

        cnn1 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(model_input)
        cnn2 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn1)
        pool1 = MaxPooling1D(pool_size=(7))(cnn2)
        drop1 = Dropout(0.25)(pool1)

#         drop1 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop1)
        
        cnn3 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop1)
        cnn4 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn3)
        pool2 = MaxPooling1D(pool_size=(6))(cnn4)
        drop2 = Dropout(0.25)(pool2)
        
#         drop2 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop2)
        
        cnn5 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop2)
        cnn6 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn5)
        pool3 = MaxPooling1D(pool_size=(6))(cnn6)
        drop3 = Dropout(0.25)(pool3)
        
#         print drop3
        feature_list.append(drop3)
        
    concat = Concatenate(axis=1)(feature_list)
    print concat
    
    lstm_out1 = LSTM(64,return_sequences=True)(concat)
    
#     flatten1 = Attention_layer(lstm_out1)
    
    flatten1 = Flatten()(lstm_out1)
    
    x = Dense(nb_classes, kernel_initializer='glorot_uniform', activation='softmax')(flatten1)

    densenet = Model(inputs=model_input, outputs=x)

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 3640
CHANNELS = 1
nb_classes = 2
batch_size = 32
nb_epoch = 40
img_dim = (ROWS*COLS,CHANNELS)

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim)

sgd = SGD(lr=0.005, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, verbose=1, mode='auto')
xtrain = X_train.reshape(-1,12*3640,1)
xvalid = X_valid.reshape(-1,12*3640,1)
model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])

Tensor("concatenate_269/concat:0", shape=(?, 338, 30), dtype=float32)
Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 429s 27ms/step - loss: 0.6121 - acc: 0.6426 - val_loss: 0.4704 - val_acc: 0.8307
Epoch 2/50
15658/15658 [==============================] - 412s 26ms/step - loss: 0.4764 - acc: 0.7688 - val_loss: 0.4895 - val_acc: 0.8622
Epoch 3/50
15658/15658 [==============================] - 416s 27ms/step - loss: 0.4344 - acc: 0.7980 - val_loss: 0.4776 - val_acc: 0.8700
Epoch 4/50
15658/15658 [==============================] - 414s 26ms/step - loss: 0.3993 - acc: 0.8209 - val_loss: 0.4889 - val_acc: 0.8762
Epoch 5/50
15658/15658 [==============================] - 408s 26ms/step - loss: 0.3717 - acc: 0.8396 - val_loss: 0.4835 - val_acc: 0.8908
Epoch 6/50
15658/15658 [==============================] - 417s 27ms/step - loss: 0.3597 - acc: 0.8463 - val_loss: 0.4740 - val_acc: 0.8945
Epoch 7/50
15658/15658 [=======================

In [26]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils

def cut(x, index):
    x = x[:,index]
    return x

def createDenseNet(nb_classes, img_dim):

    feature_list = []
    
    model_input = Input(shape=img_dim)
    
    for i in range(2):
#         inputs = Lambda(cut, arguments={'index':i})(model_input)
#         print inputs
#         print model_input

        cnn1 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(model_input)
        cnn2 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn1)
        pool1 = MaxPooling1D(pool_size=(7))(cnn2)
        drop1 = Dropout(0.25)(pool1)

#         drop1 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop1)
        
        cnn3 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop1)
        cnn4 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn3)
        pool2 = MaxPooling1D(pool_size=(6))(cnn4)
        drop2 = Dropout(0.25)(pool2)
        
#         drop2 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop2)
        
        cnn5 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop2)
        cnn6 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn5)
        pool3 = MaxPooling1D(pool_size=(6))(cnn6)
        drop3 = Dropout(0.25)(pool3)
        
#         print drop3
        feature_list.append(drop3)
        
    concat = Concatenate(axis=2)(feature_list)
    print concat
    
    lstm_out1 = LSTM(64,return_sequences=True)(concat)
    
#     flatten1 = Attention_layer(lstm_out1)
    
    flatten1 = Flatten()(lstm_out1)
    
    x = Dense(nb_classes, kernel_initializer='glorot_uniform', activation='softmax')(flatten1)

    densenet = Model(inputs=model_input, outputs=x)

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 3640
CHANNELS = 1
nb_classes = 2
batch_size = 32
nb_epoch = 40
img_dim = (ROWS*COLS,CHANNELS)

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim)

sgd = SGD(lr=0.005, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, verbose=1, mode='auto')
xtrain = X_train.reshape(-1,12*3640,1)
xvalid = X_valid.reshape(-1,12*3640,1)
model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])

Tensor("concatenate_270/concat:0", shape=(?, 169, 60), dtype=float32)
Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 289s 18ms/step - loss: 0.5975 - acc: 0.6592 - val_loss: 0.5557 - val_acc: 0.7522
Epoch 2/50
15658/15658 [==============================] - 276s 18ms/step - loss: 0.4691 - acc: 0.7760 - val_loss: 0.5070 - val_acc: 0.8680
Epoch 3/50
15658/15658 [==============================] - 282s 18ms/step - loss: 0.4291 - acc: 0.8023 - val_loss: 0.4427 - val_acc: 0.8602
Epoch 4/50
15658/15658 [==============================] - 282s 18ms/step - loss: 0.4036 - acc: 0.8198 - val_loss: 0.4648 - val_acc: 0.8808
Epoch 5/50
15658/15658 [==============================] - 281s 18ms/step - loss: 0.3848 - acc: 0.8295 - val_loss: 0.4018 - val_acc: 0.8797
Epoch 6/50
15658/15658 [==============================] - 282s 18ms/step - loss: 0.3695 - acc: 0.8385 - val_loss: 0.4047 - val_acc: 0.8995
Epoch 7/50
15658/15658 [=======================

In [37]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils

class Attention_layer(Layer): 
    def __init__(self,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')
 
        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)
 
        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)
 
        self.bias = bias
        super(Attention_layer, self).__init__(**kwargs)
 
    def build(self, input_shape):
        assert len(input_shape) == 3
 
        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
 
        super(Attention_layer, self).build(input_shape)
 
    def compute_mask(self, input, input_mask=None):
        return None
 
    def call(self, x, mask=None):
        uit = K.dot(x, self.W)
        if self.bias:
            uit += self.b
        uit = K.tanh(uit)
        a = K.exp(uit)
        if mask is not None:
            a *= K.cast(mask, K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        print a
        print x
        weighted_input = x * a
        print weighted_input
        return K.sum(weighted_input, axis=1)
 
    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

def cut(x, index):
    x = x[:,index]
    return x

def createDenseNet(nb_classes, img_dim):

    feature_list = []
    
    model_input = Input(shape=img_dim)
    
    for i in range(2):
#         inputs = Lambda(cut, arguments={'index':i})(model_input)
#         print inputs
#         print model_input

        cnn1 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(model_input)
        cnn2 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn1)
        pool1 = MaxPooling1D(pool_size=(7))(cnn2)
        drop1 = Dropout(0.25)(pool1)

#         drop1 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop1)
        
        cnn3 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop1)
        cnn4 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn3)
        pool2 = MaxPooling1D(pool_size=(6))(cnn4)
        drop2 = Dropout(0.25)(pool2)
        
#         drop2 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop2)
        
        cnn5 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop2)
        cnn6 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn5)
        pool3 = MaxPooling1D(pool_size=(6))(cnn6)
        drop3 = Dropout(0.25)(pool3)
        
#         print drop3
        feature_list.append(drop3)
        
    concat = Concatenate(axis=2)(feature_list)
    print concat
    
    lstm_out1 = LSTM(64,return_sequences=True)(concat)
    print lstm_out1
    
    flatten1 = Attention_layer()(lstm_out1)

#     flatten1 = Flatten()(lstm_out1)
    
    x = Dense(nb_classes, kernel_initializer='glorot_uniform', activation='softmax')(flatten1)

    densenet = Model(inputs=model_input, outputs=x)

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 3640
CHANNELS = 1
nb_classes = 2
batch_size = 32
nb_epoch = 40
img_dim = (ROWS*COLS,CHANNELS)

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim)

sgd = SGD(lr=0.005, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, verbose=1, mode='auto')
xtrain = X_train.reshape(-1,12*3640,1)
xvalid = X_valid.reshape(-1,12*3640,1)
model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])

Tensor("concatenate_281/concat:0", shape=(?, 169, 60), dtype=float32)
Tensor("lstm_18/transpose_1:0", shape=(?, ?, 64), dtype=float32)
Tensor("attention_layer_10/div:0", shape=(?, 169, 64), dtype=float32)
Tensor("lstm_18/transpose_1:0", shape=(?, ?, 64), dtype=float32)
Tensor("attention_layer_10/mul:0", shape=(?, 169, 64), dtype=float32)
Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 357s 23ms/step - loss: 0.6843 - acc: 0.5393 - val_loss: 0.6439 - val_acc: 0.6643
Epoch 2/50
15658/15658 [==============================] - 325s 21ms/step - loss: 0.6268 - acc: 0.6360 - val_loss: 0.5360 - val_acc: 0.7758
Epoch 3/50
15658/15658 [==============================] - 327s 21ms/step - loss: 0.5172 - acc: 0.7419 - val_loss: 0.5028 - val_acc: 0.8150
Epoch 4/50
15658/15658 [==============================] - 328s 21ms/step - loss: 0.4672 - acc: 0.7739 - val_loss: 0.4911 - val_acc: 0.8467
Epoch 5/50
15658/15658 [==============================]

In [5]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils

def cut(x, index):
    x = x[:,index]
    return x

def createDenseNet(nb_classes, img_dim):

    feature_list = []
    
    model_input = Input(shape=img_dim)
    
    for i in range(2):
#         inputs = Lambda(cut, arguments={'index':i})(model_input)
#         print inputs
#         print model_input

        cnn1 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(model_input)
        cnn2 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn1)
        pool1 = MaxPooling1D(pool_size=(7))(cnn2)
        drop1 = Dropout(0.25)(pool1)

#         drop1 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop1)
        
        cnn3 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop1)
        cnn4 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn3)
        pool2 = MaxPooling1D(pool_size=(6))(cnn4)
        drop2 = Dropout(0.25)(pool2)
        
#         drop2 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop2)
        
        cnn5 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop2)
        cnn6 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn5)
        pool3 = MaxPooling1D(pool_size=(6))(cnn6)
        drop3 = Dropout(0.25)(pool3)
        
#         print drop3
        feature_list.append(drop3)
        
    concat = Concatenate(axis=2)(feature_list)
    print concat
    
    lstm_out1 = Bidirectional(LSTM(64,return_sequences=True))(concat)
    
#     flatten1 = Attention_layer(lstm_out1)
    
    flatten1 = Flatten()(lstm_out1)
    
    x = Dense(nb_classes, kernel_initializer='glorot_uniform', activation='softmax')(flatten1)

    densenet = Model(inputs=model_input, outputs=x)

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 3640
CHANNELS = 1
nb_classes = 2
batch_size = 32
nb_epoch = 40
img_dim = (ROWS*COLS,CHANNELS)

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim)

sgd = SGD(lr=0.005, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, verbose=1, mode='auto')
xtrain = X_train.reshape(-1,12*3640,1)
xvalid = X_valid.reshape(-1,12*3640,1)
model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])

/usr/local/lib/python2.7/dist-packages/keras/activations.py:211: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


Tensor("concatenate_1/concat:0", shape=(?, 169, 60), dtype=float32)
Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 400s 26ms/step - loss: 0.5868 - acc: 0.6702 - val_loss: 0.5257 - val_acc: 0.8240
Epoch 2/50
15658/15658 [==============================] - 385s 25ms/step - loss: 0.4574 - acc: 0.7845 - val_loss: 0.5044 - val_acc: 0.8790
Epoch 3/50
15658/15658 [==============================] - 385s 25ms/step - loss: 0.4181 - acc: 0.8126 - val_loss: 0.4078 - val_acc: 0.8785
Epoch 4/50
15658/15658 [==============================] - 384s 25ms/step - loss: 0.3869 - acc: 0.8313 - val_loss: 0.4251 - val_acc: 0.8928
Epoch 5/50
15658/15658 [==============================] - 383s 24ms/step - loss: 0.3670 - acc: 0.8444 - val_loss: 0.3778 - val_acc: 0.9005
Epoch 6/50
15658/15658 [==============================] - 385s 25ms/step - loss: 0.3561 - acc: 0.8492 - val_loss: 0.3733 - val_acc: 0.9038
Epoch 7/50
15658/15658 [=========================

In [14]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils

def cut(x, index):
    x = x[:,index]
    return x

def createDenseNet(nb_classes, img_dim):

    feature_list = []
    
    model_input = Input(shape=img_dim)
    
    for i in range(2):
#         inputs = Lambda(cut, arguments={'index':i})(model_input)
#         print inputs
#         print model_input

        cnn1 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(model_input)
        cnn2 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn1)
        pool1 = MaxPooling1D(pool_size=(7))(cnn2)
        drop1 = Dropout(0.25)(pool1)

#         drop1 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop1)
        
        cnn3 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop1)
        cnn4 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn3)
        pool2 = MaxPooling1D(pool_size=(6))(cnn4)
        drop2 = Dropout(0.25)(pool2)
        
#         drop2 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop2)
        
        cnn5 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop2)
        cnn6 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn5)
        pool3 = MaxPooling1D(pool_size=(6))(cnn6)
        drop3 = Dropout(0.25)(pool3)
        
#         print drop3
        feature_list.append(drop3)
        
    concat = Concatenate(axis=2)(feature_list)
    print concat
    
    lstm_out1 = LSTM(64,return_sequences=True)(concat)
    
#     flatten1 = Attention_layer(lstm_out1)
    
    flatten1 = Flatten()(lstm_out1)
    
    x = Dense(nb_classes, kernel_initializer='glorot_uniform', activation='softmax')(flatten1)

    densenet = Model(inputs=model_input, outputs=x)

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 3640
CHANNELS = 1
nb_classes = 2
batch_size = 32
nb_epoch = 40
img_dim = (ROWS*COLS,CHANNELS)

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim)

sgd = SGD(lr=0.005, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, verbose=1, mode='auto')
model_checkpoint = keras.callbacks.ModelCheckpoint('model.h5', monitor="val_acc", save_best_only=True,save_weights_only=True, verbose=1)

xtrain = X_train.reshape(-1,12*3640,1)
xvalid = X_valid.reshape(-1,12*3640,1)
model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop,model_checkpoint])

Tensor("concatenate_10/concat:0", shape=(?, 169, 60), dtype=float32)
Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 291s 19ms/step - loss: 0.6134 - acc: 0.6389 - val_loss: 0.5179 - val_acc: 0.7525

Epoch 00001: val_acc improved from -inf to 0.75250, saving model to model.h5
Epoch 2/50
15658/15658 [==============================] - 283s 18ms/step - loss: 0.4745 - acc: 0.7666 - val_loss: 0.4506 - val_acc: 0.8482

Epoch 00002: val_acc improved from 0.75250 to 0.84825, saving model to model.h5
Epoch 3/50
15658/15658 [==============================] - 284s 18ms/step - loss: 0.4330 - acc: 0.7987 - val_loss: 0.3936 - val_acc: 0.8810

Epoch 00003: val_acc improved from 0.84825 to 0.88100, saving model to model.h5
Epoch 4/50
15658/15658 [==============================] - 284s 18ms/step - loss: 0.4052 - acc: 0.8186 - val_loss: 0.4071 - val_acc: 0.8840

Epoch 00004: val_acc improved from 0.88100 to 0.88400, saving model to model.h5
Epoch

In [24]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils

def cut(x, index):
    x = x[:,index]
    return x

def createDenseNet(nb_classes, img_dim):

    feature_list = []
    
    model_input = Input(shape=img_dim)
    
    for i in range(2):
#         inputs = Lambda(cut, arguments={'index':i})(model_input)
#         print inputs
#         print model_input

        cnn1 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(model_input)
        cnn2 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn1)
        pool1 = MaxPooling1D(pool_size=(7))(cnn2)
        drop1 = Dropout(0.25)(pool1)

#         drop1 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop1)
        
        cnn3 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop1)
        cnn4 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn3)
        pool2 = MaxPooling1D(pool_size=(6))(cnn4)
        drop2 = Dropout(0.25)(pool2)
        
#         drop2 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop2)
        
        cnn5 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop2)
        cnn6 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn5)
        pool3 = MaxPooling1D(pool_size=(6))(cnn6)
        drop3 = Dropout(0.25)(pool3)
        
#         print drop3
        feature_list.append(drop3)
        
    concat = Concatenate(axis=2)(feature_list)
    print concat
    
    lstm_out1 = LSTM(64,return_sequences=True)(concat)
    
#     flatten1 = Attention_layer(lstm_out1)
    
    flatten1 = Flatten()(lstm_out1)
    
    x = Dense(nb_classes, kernel_initializer='glorot_uniform', activation='softmax')(flatten1)

    densenet = Model(inputs=model_input, outputs=x)

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 3640
CHANNELS = 1
nb_classes = 2
batch_size = 32
nb_epoch = 40
img_dim = (ROWS*COLS,CHANNELS)

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim)

sgd = SGD(lr=0.005, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, verbose=1, mode='auto')
model_checkpoint = keras.callbacks.ModelCheckpoint('model.h5', monitor="val_acc", save_best_only=True,save_weights_only=True, verbose=1)

xtrain = X_train.reshape(-1,12*3640,1)
xvalid = X_valid.reshape(-1,12*3640,1)
model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop,model_checkpoint])

Tensor("concatenate_20/concat:0", shape=(?, 169, 60), dtype=float32)
Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 290s 19ms/step - loss: 0.6106 - acc: 0.6484 - val_loss: 0.4968 - val_acc: 0.7775

Epoch 00001: val_acc improved from -inf to 0.77750, saving model to model.h5
Epoch 2/50
15658/15658 [==============================] - 283s 18ms/step - loss: 0.4702 - acc: 0.7727 - val_loss: 0.4317 - val_acc: 0.8558

Epoch 00002: val_acc improved from 0.77750 to 0.85575, saving model to model.h5
Epoch 3/50
15658/15658 [==============================] - 285s 18ms/step - loss: 0.4234 - acc: 0.8068 - val_loss: 0.4516 - val_acc: 0.8825

Epoch 00003: val_acc improved from 0.85575 to 0.88250, saving model to model.h5
Epoch 4/50
15658/15658 [==============================] - 283s 18ms/step - loss: 0.3990 - acc: 0.8212 - val_loss: 0.5414 - val_acc: 0.8445

Epoch 00004: val_acc did not improve from 0.88250
Epoch 5/50
15658/15658 [===========